In [1]:
import tkinter as tk
from tkinter import messagebox
import numpy as np
import random
import time


# Backtracking Solver
class KenKenSolverBacktracking:
    def __init__(self, grid_size, cages, callback=None, delay=0):
        self.grid_size = grid_size
        self.cages = cages
        self.grid = np.zeros((grid_size, grid_size), dtype=int)
        self.callback = callback
        self.delay = delay

    def is_valid(self, row, col, num):
        if num in self.grid[row]:  # Check if the number is already in the row
            return False
        if num in self.grid[:, col]:  # Check if the number is already in the column
            return False

        for cage in self.cages:
            if (row, col) in cage["cells"]:
                cage_nums = [
                    self.grid[r][c]
                    for r, c in cage["cells"]
                    if self.grid[r][c] != 0
                ]
                if not self.check_cage(cage, cage_nums + [num]):
                    return False
        return True

    def check_cage(self, cage, nums):
        target, operation = cage["target"], cage["operation"]
        if len(nums) > len(cage["cells"]):
            return False
        if operation == "+":
            return sum(nums) == target if len(nums) == len(cage["cells"]) else sum(nums) <= target
        elif operation == "-":
            if len(nums) == len(cage["cells"]):
                return abs(nums[0] - nums[1]) == target
            return True
        elif operation == "*":
            product = 1
            for num in nums:
                product *= num
            return product == target if len(nums) == len(cage["cells"]) else product <= target
        elif operation == "/":
            if len(nums) == len(cage["cells"]):
                return max(nums) % min(nums) == 0 and max(nums) // min(nums) == target
            return True
        return False

    def solve(self, row=0, col=0):
        if row == self.grid_size:
            return True

        next_row, next_col = (row + 1, 0) if col == self.grid_size - 1 else (row, col + 1)

        if self.grid[row][col] != 0:
            return self.solve(next_row, next_col)

        for num in range(1, self.grid_size + 1):
            if self.is_valid(row, col, num):
                self.grid[row][col] = num
                if self.callback:
                    self.callback(self.grid, row, col)
                    time.sleep(self.delay)
                if self.solve(next_row, next_col):
                    return True
                self.grid[row][col] = 0
                if self.callback:
                    self.callback(self.grid, row, col)
                    time.sleep(self.delay)

        return False


# Genetic Algorithm Solver
class KenKenSolverGenetic:
    def __init__(self, grid_size, cages, callback=None, delay=0.1, population_size=100, generations=1000):
        self.grid_size = grid_size
        self.cages = cages
        self.population_size = population_size
        self.generations = generations
        self.population = self.initialize_population()
        self.callback = callback
        self.delay = delay

    def initialize_population(self):
        population = []
        base_row = list(range(1, self.grid_size + 1))
        for _ in range(self.population_size):
            grid = np.array([random.sample(base_row, self.grid_size) for _ in range(self.grid_size)])
            population.append(grid)
        return population

    def fitness(self, grid):
        """Calculate fitness based on uniqueness and cage satisfaction."""
        score = 0

        # Row and column uniqueness
        for row in grid:
            score += len(set(row))  # Count unique values in the row
        for col in grid.T:
            score += len(set(col))  # Count unique values in the column

        # Cage constraints
        for cage in self.cages:
            cage_values = [grid[r][c] for r, c in cage["cells"]]
            if self.check_cage(cage, cage_values):
                score += 10  # Assign higher weight for satisfying cage constraints

        return score

    def check_cage(self, cage, nums):
        """Validate if a cage satisfies its constraints."""
        target, operation = cage["target"], cage["operation"]
        if operation == "+":
            return sum(nums) == target
        elif operation == "-":
            return abs(nums[0] - nums[1]) == target
        elif operation == "*":
            product = 1
            for num in nums:
                product *= num
            return product == target
        elif operation == "/":
            return max(nums) % min(nums) == 0 and max(nums) // min(nums) == target
        return False

    def crossover(self, parent1, parent2):
        """Perform crossover to produce a valid child."""
        child = parent1.copy()
        for i in range(self.grid_size):
            if random.random() > 0.5:  # Randomly choose rows from each parent
                child[i] = parent2[i]
        return self.ensure_valid_grid(child)

    def mutate(self, grid):
        """Perform mutation by swapping two elements in a row."""
        row = random.randint(0, self.grid_size - 1)
        col1, col2 = random.sample(range(self.grid_size), 2)  # Choose two random columns
        grid[row][col1], grid[row][col2] = grid[row][col2], grid[row][col1]  # Swap values
        return self.ensure_valid_grid(grid)

    def ensure_valid_grid(self, grid):
        """Ensure the grid has unique rows and columns."""
        for i in range(self.grid_size):
            row = grid[i]
            missing_numbers = set(range(1, self.grid_size + 1)) - set(row)
            duplicates = [num for num in row if list(row).count(num) > 1]

            for j, num in enumerate(row):
                if num in duplicates:
                    if missing_numbers:
                        row[j] = missing_numbers.pop()
                    duplicates.remove(num)

        for j in range(self.grid_size):
            col = grid[:, j]
            missing_numbers = set(range(1, self.grid_size + 1)) - set(col)
            duplicates = [num for num in col if list(col).count(num) > 1]

            for i, num in enumerate(col):
                if num in duplicates:
                    if missing_numbers:
                        grid[i, j] = missing_numbers.pop()
                    duplicates.remove(num)

        return grid

    def evolve(self):
        """Run the genetic algorithm."""
        best_individual = None
        best_fitness = 0

        for generation in range(self.generations):
            fitness_scores = [self.fitness(grid) for grid in self.population]
            max_fitness = max(fitness_scores)
            best_individual = self.population[fitness_scores.index(max_fitness)]
            best_fitness = max(best_fitness, max_fitness)

            # Exit condition: fitness threshold (approaching perfect solution)
            if best_fitness == self.grid_size * (self.grid_size * 2 + len(self.cages) * 10):
                break

            sorted_population = [
                grid
                for _, grid in sorted(
                    zip(fitness_scores, self.population), key=lambda x: x[0], reverse=True
                )
            ]
            self.population = sorted_population[: self.population_size // 2]
            new_population = []

            for i in range(0, len(self.population), 2):
                parent1 = self.population[i]
                parent2 = self.population[(i + 1) % len(self.population)]
                child = self.crossover(parent1, parent2)
                if random.random() < 0.1:  # Mutation probability
                    child = self.mutate(child)
                new_population.append(child)

            self.population += new_population

            if self.callback and generation % 10 == 0:  # Update visualization every 10 generations
                self.callback(best_individual)
                time.sleep(self.delay)

        return best_individual
class KenKenSolverApp:
    def __init__(self, root):
        self.root = root
        self.root.title("KenKen Puzzle Solver")
        self.grid_size = 6
        self.cages = []
        self.grid_entries = []

        self.control_frame = tk.Frame(self.root)
        self.grid_frame = tk.Frame(self.root)
        self.log_frame = tk.Frame(self.root)

        self.control_frame.pack(pady=10)
        self.grid_frame.pack(pady=10)
        self.log_frame.pack(pady=10)

        tk.Label(self.control_frame, text="Grid Size:").grid(row=0, column=0, padx=5)
        self.grid_size_entry = tk.Entry(self.control_frame, width=5)
        self.grid_size_entry.insert(0, str(self.grid_size))
        self.grid_size_entry.grid(row=0, column=1, padx=5)

        tk.Button(self.control_frame, text="Change Grid Size", command=self.change_grid_size).grid(row=0, column=2, padx=5)
        tk.Button(self.control_frame, text="Add Cage", command=self.add_cage).grid(row=0, column=3, padx=5)
        tk.Button(self.control_frame, text="Solve with Backtracking", command=self.solve_with_backtracking).grid(row=0, column=4, padx=5)
        tk.Button(self.control_frame, text="Observe Backtracking", command=self.observe_backtracking).grid(row=0, column=5, padx=5)
        tk.Button(self.control_frame, text="Solve with Genetic", command=self.solve_with_genetic).grid(row=0, column=6, padx=5)
        tk.Button(self.control_frame, text="Observe Genetic", command=self.observe_genetic).grid(row=0, column=7, padx=5)
        tk.Button(self.control_frame, text="Clear Grid", command=self.clear_grid).grid(row=0, column=8, padx=5)

        self.log_box = tk.Text(self.log_frame, height=10, width=80, state="disabled")
        self.log_box.pack()

        self.initialize_grid()

    def initialize_grid(self):
        for widget in self.grid_frame.winfo_children():
            widget.destroy()
        self.grid_entries = []
        for row in range(self.grid_size):
            row_entries = []
            for col in range(self.grid_size):
                entry = tk.Entry(
                    self.grid_frame, width=5, font=("Arial", 18), justify="center"
                )
                entry.grid(row=row, column=col, padx=5, pady=5)
                row_entries.append(entry)
            self.grid_entries.append(row_entries)

    def change_grid_size(self):
        try:
            size = int(self.grid_size_entry.get())
            if size < 2 or size > 12:
                raise ValueError("Grid size must be between 2 and 12.")
            self.grid_size = size
            self.cages = []
            self.initialize_grid()
            self.log_message(f"Grid size changed to {size}x{size}.")
        except ValueError as e:
            messagebox.showerror("Error", str(e))

    def add_cage(self):
        cage_window = tk.Toplevel(self.root)
        cage_window.title("Add Cage")

        tk.Label(cage_window, text="Target:").grid(row=0, column=0)
        target_entry = tk.Entry(cage_window)
        target_entry.grid(row=0, column=1)

        tk.Label(cage_window, text="Operation (+, -, *, /):").grid(row=1, column=0)
        operation_entry = tk.Entry(cage_window)
        operation_entry.grid(row=1, column=1)

        tk.Label(cage_window, text="Cells (e.g., 0,0 1,1):").grid(row=2, column=0)
        cells_entry = tk.Entry(cage_window)
        cells_entry.grid(row=2, column=1)

        def save_cage():
            try:
                target = int(target_entry.get())
                operation = operation_entry.get().strip()
                cells = [
                    tuple(map(int, cell.split(",")))
                    for cell in cells_entry.get().split()
                ]
                if operation not in ["+", "-", "*", "/"]:
                    raise ValueError("Invalid operation.")
                for r, c in cells:
                    if r < 0 or c < 0 or r >= self.grid_size or c >= self.grid_size:
                        raise ValueError("Cell coordinates out of bounds.")
                self.cages.append({"target": target, "operation": operation, "cells": cells})
                self.log_message(f"Added cage: Target={target}, Operation={operation}, Cells={cells}")
                cage_window.destroy()
            except Exception as e:
                messagebox.showerror("Error", str(e))

        tk.Button(cage_window, text="Save", command=save_cage).grid(row=3, column=0, columnspan=2, pady=10)

    def observe_backtracking(self):
        solver = KenKenSolverBacktracking(self.grid_size, self.cages, callback=self.update_grid, delay=0.1)
        self.log_message("Observing Backtracking...")
        start_time = time.time()
        if solver.solve():
            elapsed_time = time.time() - start_time
            self.log_message(f"Solved successfully in {elapsed_time:.2f} seconds!")
        else:
            elapsed_time = time.time() - start_time
            messagebox.showinfo("Result", f"No solution found in {elapsed_time:.2f} seconds.")

    def observe_genetic(self):
        solver = KenKenSolverGenetic(self.grid_size, self.cages, callback=self.update_grid, delay=0.1)
        self.log_message("Observing Genetic Algorithm...")
        start_time = time.time()
        solution = solver.evolve()
        elapsed_time = time.time() - start_time
        if solution is not None:
            self.update_grid(solution)
            self.log_message(f"Solved successfully using Genetic Algorithm in {elapsed_time:.2f} seconds!")
        else:
            messagebox.showinfo("Result", f"No solution found in {elapsed_time:.2f} seconds.")

    def solve_with_backtracking(self):
        solver = KenKenSolverBacktracking(self.grid_size, self.cages)
        self.log_message("Solving with Backtracking...")
        start_time = time.time()
        if solver.solve():
            elapsed_time = time.time() - start_time
            self.update_grid(solver.grid)
            self.log_message(f"Solved successfully in {elapsed_time:.2f} seconds!")
        else:
            elapsed_time = time.time() - start_time
            messagebox.showinfo("Result", f"No solution found in {elapsed_time:.2f} seconds.")

    def solve_with_genetic(self):
        solver = KenKenSolverGenetic(self.grid_size, self.cages)
        self.log_message("Solving with Genetic Algorithm...")
        start_time = time.time()
        solution = solver.evolve()
        elapsed_time = time.time() - start_time
        if solution is not None:
            self.update_grid(solution)
            self.log_message(f"Solved successfully using Genetic Algorithm in {elapsed_time:.2f} seconds!")
        else:
            messagebox.showinfo("Result", f"No solution found in {elapsed_time:.2f} seconds.")

    def update_grid(self, solution, row=None, col=None):
        for i, grid_row in enumerate(solution):
            for j, num in enumerate(grid_row):
                entry = self.grid_entries[i][j]
                entry.delete(0, tk.END)
                entry.insert(0, str(num))
                if row == i and col == j:
                    entry.config(bg="lightblue")
                else:
                    entry.config(bg="white")
        self.root.update_idletasks()

    def clear_grid(self):
        for row in self.grid_entries:
            for cell in row:
                cell.delete(0, tk.END)
        self.log_message("Grid cleared.")

    def log_message(self, message):
        self.log_box.config(state="normal")
        self.log_box.insert(tk.END, f"{message}\n")
        self.log_box.config(state="disabled")
        self.log_box.yview(tk.END)


if __name__ == "__main__":
    root = tk.Tk()
    app = KenKenSolverApp(root)
    root.mainloop()